In [ ]:
from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.python.keras.layers import Input, LSTM, Dense, Activation
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.utils import to_categorical
import numpy as np

## Dados

Carregamos todo o texto (porque dá) e passamos para minúsculas

In [ ]:
text = open('data/lusiadas.txt', encoding='utf-8').read()
text = text.lower()

Detectamos os caracteres e mapeamos os índices

In [ ]:
chars = sorted(set(text))
char_to_int = {c: to_categorical(i, len(chars)).ravel() for i, c in enumerate(chars)}

Geramos os exemplos "sequencia -> proxima letra"

In [ ]:
length = 20
x, y = [], []
word = text[:length]
for letter in text[length:]:
    x.append([char_to_int[w] for w in word])
    y.append(char_to_int[letter])
    word = word[1:] + letter
x = np.asarray(x)
y = np.asarray(y)

Embaralhando o dataset

In [ ]:
i = np.random.permutation(x.shape[0])
x, y = x[i], y[i]

Separando treino e teste

In [ ]:
i = int(.8 * len(x))
x_train, x_test = x[:i], x[i:]
y_train, y_test = y[:i], y[i:]

## Modelo

Camada de entrada (compatível com a forma de x)

In [ ]:
out = entry = Input(shape=x_train.shape[1:])

Camada de memória

In [ ]:
out = LSTM(256)(out)

Camada de saída com um neurônio para cada caractere e aplicação do softmax para obtermos uma distribuição de probabilidade

In [ ]:
out = Dense(y_train.shape[1])(out)
out = Activation('softmax')(out)

Definição do modelo em si

In [ ]:
net = Model(entry, out)

Imprimimos a descrição do modelo

In [ ]:
net.summary()

## Treinamento

Definição do custo e da otimização

In [ ]:
net.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(lr=0.002, momentum=0.9, nesterov=True),
    metrics=['accuracy'])

Treinamento em si

In [ ]:
net.fit(
    x_train, y_train,
    batch_size=512,
    epochs=10,
    validation_data=(x_test, y_test),
    callbacks=[
        ModelCheckpoint('save/text.{epoch:02d}.h5'),
        TensorBoard(log_dir='logs/text', histogram_freq=1)])

## Geração de texto

Ponto de partida

In [ ]:
text = '''As armas, & os barões aſsinalados,
Que da Occidental praya Luſitana,
Por mares nunca'''.lower()

Geração em si

In [ ]:
for _ in range(1000):
    word = text[-length:]

    x = [[char_to_int[w] for w in word]]
    y = net.predict(x, verbose=0)[0]

    char = np.random.choice(chars, p=y)
    # char = chars[np.argmax(y)]

    text += char

print(text)